### (1) Load the pretrained model

In [1]:
import sys
sys.path.insert(1, './dlite-v2-1_5b/')
from instruct_pipeline import InstructionTextGenerationPipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("dlite-v2-1_5b", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("dlite-v2-1_5b", device_map="cuda", torch_dtype=torch.bfloat16)

generate_text = InstructionTextGenerationPipeline(model=model, tokenizer=tokenizer)

### (2) The REST server

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import subprocess
import re

app = Flask(__name__)
CORS(app)

def req_length(text):
    tokens = tokenizer(text)
    tokens_length = len(tokens.attention_mask)
    print('LOG: Input length: ' + str(tokens_length))
    return tokens_length

def req_to_json(title, text):
    return jsonify({title : text})

def req_process(command, text):
    req = command + ' ' + text
    print('LOG: ' + req)
    return generate_text(req)

def req_clean(text):
    cleaned_text = text.replace('\n', ' ').replace('\r', ' ')
    cleaned_text = re.sub(r'[^\w\s.]', '', cleaned_text)
    return cleaned_text

@app.route('/sum', methods=['POST'])
def sum_req():
    data = request.get_json()
    print(data)
    text = req_clean(data.get('text', ''))
    if(req_length(text) <= 1024):
        return req_to_json('response', req_process('Summarize in one sentence:', text))
        #return req_to_json('response', "TEST TEXT")
    else:
        return req_to_json('error', 'Too long text. Limit is 1024 token.')

@app.route('/key', methods=['POST'])
def key_req():
    data = request.get_json()
    text = req_clean(data.get('text', ''))
    if(req_length(text) <= 1024):
        return req_to_json('response', req_process('Give some keywords from this text:', text))
    else:
        return req_to_json('error', 'Too long text. Limit is 1024 token.')

@app.route('/title', methods=['POST'])
def title_req():
    data = request.get_json()
    text = req_clean(data.get('text', ''))
    if(req_length(text) <= 1024):
        return req_to_json('response', req_process('Give the most important word in the text:', text))
    else:
        return req_to_json('error', 'Too long text. Limit is 1024 token.')

@app.route('/length', methods=['POST'])
def length_req():
    data = request.get_json()
    text = req_clean(data.get('text', ''))
    ret = {'length': req_length(text)}
    return jsonify(ret)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


{'text': "Jupiter, the largest planet in our solar system, commands attention with its immense size and swirling clouds of gas. Named after the king of the Roman gods, Jupiter's colossal presence dominates the space around it. Its massive magnetic field creates a protective shield, deflecting harmful solar winds and radiation, making it a formidable guardian of the inner solar system."}
LOG: Input length: 67
LOG: Summarize in one sentence: Jupiter the largest planet in our solar system commands attention with its immense size and swirling clouds of gas. Named after the king of the Roman gods Jupiters colossal presence dominates the space around it. Its massive magnetic field creates a protective shield deflecting harmful solar winds and radiation making it a formidable guardian of the inner solar system.


127.0.0.1 - - [11/May/2024 20:18:33] "POST /sum HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2024 20:19:13] "POST /length HTTP/1.1" 200 -


LOG: Input length: 249


...

LOG: Input length: 249
LOG: Give some keywords from this text: The MiG21 a legendary Sovietdesigned supersonic fighter aircraft holds a revered place in aviation history. Introduced in the 1950s it became one of the most widely produced jet fighters in the world serving with over 60 countries across several decades. Its sleek deltawing design and powerful engines gave it impressive speed and agility earning it the nickname Fishbed in NATO nomenclature.  Renowned for its simplicity and reliability the MiG21 was designed to be easy to maintain and operate making it a favorite among air forces with limited resources. Its delta wing configuration allowed for quick takeoffs and landings even from short runways making it wellsuited for operations in a variety of environments.  The MiG21 saw extensive combat action during the Cold War and in numerous conflicts around the world. Its combat record includes engagements in Vietnam the Middle East and Africa where its speed and maneuverability oft

127.0.0.1 - - [11/May/2024 20:20:03] "POST /key HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2024 20:25:52] "POST /length HTTP/1.1" 200 -


LOG: Input length: 242


127.0.0.1 - - [11/May/2024 20:27:57] "POST /length HTTP/1.1" 200 -


LOG: Input length: 242
LOG: Input length: 242
LOG: Give some keywords from this text: Cats beloved companions to millions around the world captivate us with their grace independence and mysterious allure. Domesticated for thousands of years these enigmatic creatures have woven themselves into the fabric of human society earning their place as cherished pets and valued members of the family.  Known for their agility and keen hunting instincts cats are skilled predators capable of navigating even the most precarious of environments with ease. Their lithe bodies sharp claws and acute senses make them formidable hunters capable of stalking prey with silent precision.  Yet despite their predatory nature cats also possess a softer side. They are affectionate animals forming strong bonds with their human companions and often seeking out cuddles and attention. Their soothing purrs and gentle nuzzles bring comfort and joy to those fortunate enough to share their homes.  Cats come in a variety o